In [1]:
from libs.Dataset import *
from libs.SiameseNetwork import *
import matplotlib.pyplot as plt
import dill # allow to save notebook session, useful for test https://towardsdatascience.com/how-to-restore-your-jupyter-notebook-session-dfeadbd86d65
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import Callback, progress
from torch.optim import SGD
from torch import nn
import torch


In [2]:
# load datamodule sessions
dm = "datamodule"
dill.load_session('notebook_env_28.db')

In [3]:
print("Dims datamodel: {}".format(dm.dims))

Dims datamodel: (3, 28, 28)


/Users/danilo/GitHub/deep-learning/venv/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:170: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")


### Valuto prestazioni con modelli prima del training

In [4]:
from torchvision.models import mobilenet_v2
from torchvision.models import squeezenet1_1

# uso mobileNetV2 come primo esempio
mobilenet_fe = mobilenet_v2(pretrained=True)
mobilenet_fe.features[0] = nn.Conv2d(3,32,5)
# rimuovo classificatore finale, imposto il modulo identità al posto del classificatore
mobilenet_fe.classifier = nn.Identity()


squezenet_fe = squeezenet1_1(pretrained=True)
# rimuovo classificatore finale, imposto il modulo identità al posto del classificatore
squezenet_fe.classifier = nn.Identity()



print("SqueezeNet1_1: Dim feature vector for:  {} -> {}".format(torch.zeros(1,3,224,224).shape, squezenet_fe(torch.zeros(1,3,224,224)).shape))
print("MobileNetV2: Dim feature vector for: {} -> {}".format(torch.zeros(1,3,224,224).shape, mobilenet_fe(torch.zeros(1,3,224,224)).shape))

/Users/danilo/GitHub/deep-learning/venv/lib/python3.9/site-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


SqueezeNet1_1: Dim feature vector for:  torch.Size([1, 3, 224, 224]) -> torch.Size([1, 86528])
MobileNetV2: Dim feature vector for: torch.Size([1, 3, 224, 224]) -> torch.Size([1, 1280])


In [5]:
from tqdm import tqdm


def extract_rgb_rep(loader):
    """ Baseline che effettua ricerca nearest neighbor sui valori RGB grezzi.
    (ricerca l'immagine visivamente più simile)"""
    representations, labels = [], []
    for batch in tqdm(loader, total=len(loader)):
        print(batch.size())
        print(batch.shape())
        print(batch)
        representations.append(batch[0].view(batch[0].shape[0],-1).numpy())
        labels.append(batch[1])

    return np.concatenate(representations), np.concatenate(labels)


sample_dl = dm.val_dataloader()

repres, labels = extract_rgb_rep(sample_dl)


  0%|          | 0/83 [00:00<?, ?it/s]